# Randomness and stochastic music

This notebook gives an overview of musx tools for working with randomness.

Running this notebook requires the musx package. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
import math
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/taube/Software/musx')
from musx import Score, Note, Seq, MidiFile, choose, scale, jumble, intempo, \
    odds, pick, between, setmidiplayer, version, playfile, interp, rescale, \
    uniran, lowran, midran, highran, beta, gauss, white, pink, brown, vary, spray
from musx.paint import spray
print(f"musx version: {version}")

Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print('OK!')

## Discrete vs continuous random selection

In music composition, random processes are used to generate both continuous and descrete musical outcomes. In this notebook floating point values used for continuous values and integers for discrete choices. For example, amplitude and hertz values (frequency) are continuous since any change in hertz produces a unique frequency. In contrast, equal tempered pitch and midi key numbers produce discrete (specific) pitchs. Musx provide a number of different random number generators, each with a unique *probability distribution* that determines its possible outcomes. The following sections demonstrate examples of  distrubute sonified to descrete and continuous results.

Setup for graphing and performing distributions. Note: since standard midi files are inherently discrete, the midi performances of continuous results use channel tuning to quantize hertz to ~12 cent increments (100¢ / 9 = 12.5)).  The mode parameter to the discrete() composer allows experimentation with variety of musical scales, all of which start on keynum 21 and end on keynum 107.

In [ ]:
# midi microtuning
meta = MidiFile.metatrack(microdivs=9)

def continuous(score, num, rans, mini, maxi, dur, amp):
    for i in range(num):
        r = rans[i]
        k = rescale(r, mini, maxi, 21, 106)
        m = Note(time=score.now, duration=dur, pitch=k, amplitude=amp)
        score.add(m)
        yield dur

def playcontinuous(data, a=0, b=1):
    score = Score(out=Seq())
    score.compose(continuous(score, 80, data, a, b, .12, .7))
    file = MidiFile("randomness.mid", [meta, score.out]).write()
    print(f"Wrote '{file.pathname}'.")
    playfile(file.pathname) 

def discrete(score, num, rans, mini, maxi, dur, amp, mode):
    if mode == 1:    # pentatonic
        gamut = scale(21, 37, 2,2,3,2,3)
    elif mode == 2:  # blues
        gamut = scale(21, 6*7+2, 3,2,1,1,3,2)
    elif mode == 3:  # hexatonic
        gamut = scale(21, 6*7+2, 1,3)    
    elif mode == 4:  # octatonic
        gamut = scale(21, 8*7+2, 2,1)
    else:            # chromatic
        gamut = [k for k in range(21, 107)]
        raise ValueError(f"mode {mode} not 1,2,3, or 4.")
    for i in range(num):
        r = rans[i]
        h = rescale(r, mini, maxi, 0, len(gamut))
        k = gamut[int(h)]
        m = Note(time=score.now, duration=dur, pitch=k, amplitude=amp)
        score.add(m)
        yield dur

def playdiscrete(data, a=0, b=1, mode=1):
    score = Score(out=Seq())
    score.compose(discrete(score, 80, data, a, b, .12, .7, mode))
    f = MidiFile("discrete.mid", [meta, score.out]).write()
    playfile(f.pathname)
    
def plot(data):
    plt.plot(data)
    plt.show()
        
def histogram(data):
    plt.hist(data, bins=30, facecolor="blue", alpha=0.5) 
    plt.show()

print('OK!')

### Uniform distribution

```uniran()```

Returns uniform random numbers between 0.0 and 1.0 (exclusive).

In [ ]:
data = [uniran() for _ in range(5000)]
histogram(data)

Listen to continuous uniform randomness:

In [ ]:
playcontinuous(data)

Listen to discrete uniform randomness:

In [ ]:
playdiscrete(data, mode=1)

### Low-pass distribution

```lowran()```

Returns a floating point value between 0.0 and 1.0 with lower values more likely.

Histogram plot:

In [ ]:
data = [lowran() for _ in range(5000)]
histogram(data)

Listen to 'continuous' low-pass randomness:

In [ ]:
playcontinuous(data)

Listen to discrete low-pass randomness:

In [ ]:
playdiscrete(data, mode=1)

### Mid-pass distribution

```midran()```

Returns a floating point value between 0.0 and 1.0 with midrange values more likely.

Histogram plot:

In [ ]:
data = [midran() for _ in range(5000)]
histogram(data)

Listen to 'continuous' mid-range randomness:

In [ ]:
playcontinuous(data)

Listen to discrete mid-range randomness:

In [ ]:
playdiscrete(data, mode=4)

### High-pass distribution

```highran()```

Returns a floating point value between 0.0 and 1.0 with higher values more likely.

Histogram plot:

In [ ]:
data = [highran() for _ in range(5000)]
histogram(data)

Listen to 'continuous' high-pass randomness:

In [ ]:
playcontinuous(data)

Listen to discrete high-pass randomness:

In [ ]:
playdiscrete(data, mode=2)

### Beta distribution

```beta(alpha, beta)```

Returns value between 0 and 1 from the [beta distribution](https://en.wikipedia.org/wiki/Beta_distribution). When alpha=beta=1 the distribution is uniform. When alpha=beta, the distribution is symmetric around .5. When alpha<1 and beta<1 then the density of larger and smaller numbers increases. When alpha>1 and beta>1, density is similar to the gaussian distribution.

Histogram plots for different alpha and beta values (a and b):

In [ ]:
a,b = 5,5
data = [beta(a,b) for _ in range(5000)]
histogram(data)

In [ ]:
a,b = 1,1
data = [beta(a,b) for _ in range(5000)]
histogram(data)

In [ ]:
a,b = .1,.1
data = [beta() for _ in range(5000)]
histogram(data)

In [ ]:
a,b = .3,.3
data = [beta(a,b) for _ in range(5000)]
histogram(data)

Listen to 'continuous' beta randomness:

In [ ]:
playcontinuous(data)

Listen to discrete beta randomness:

In [ ]:
playdiscrete(data, mode=1)

Interpolate beta smoothly from "normal" &rarr; "uniform" &rarr; beta=.1 :

In [ ]:
def playbeta(score, num, dur, amp):
    gamut = scale(21, 37, 2,2,3,2,3) #(21, 8*7+2, 2,1)
    for i in range(num):
        c = interp(i, 0, 5, 100, .1)
        r = beta(c,c)
        h = rescale(r, 0, 1, 0, len(gamut)-1)
        k = gamut[int(h)]
        m = Note(time=score.now, duration=dur, pitch=k, amplitude=.7)
        score.add(m)
        yield dur
        
score = Score(out=Seq())
score.compose(playbeta(score, 150, .12, .7))
file = MidiFile("randomness.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

### Gaussian distribution

```gauss(*sigma*=1, *mu*=0)```

Returns unbounded value from the [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with standard deviation *sigma* and mean *mu*.  The spread (standard deviation) is 1.0 centered at 0, so 68.26% of the results are between -1 and 1 inclusive and 99.74% of the results are between -3 and 3 inclusive.

Histogram plots:

In [ ]:
data = [gauss() for _ in range(5000)]
histogram(data)

Listen to 'continuous' gaussian randomness:

In [ ]:
playcontinuous(data, -3, 3)

Listen to discrete gaussian randomness:

In [ ]:
playdiscrete(data, -3, 3)

### White noise

```white()```

Returns white (uniform noise) samples between -1.0 and 1.0.

In [ ]:
data = [white() for _ in range(5000)]
histogram(data)

Listen to 'continuous' white noise randomness:

In [ ]:
playcontinuous(data, -1, 1)

Listen to  discrete white noise randomness:

In [ ]:
playdiscrete(data, -1, 1)

### Pink noise

```pink()```

Returns pinkish (1/f) noise samples between -1.0 and 1.0. See:

* Voss RF, Clarke J (1975) [1/f noise’ in music and speech](http://123.physics.ucdavis.edu/week_3_files/voss-clarke.pdf). Nature 258(5533):317–318.
* Dan Wu,Keith M. Kendrick,Daniel J. Levitin,Chaoyi Li,Dezhong Yao (2015) [Bach Is the Father of Harmony: Revealed by a 1/f Fluctuation Analysis across Musical Genres](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0142431). PloS One. 2015;10. doi: e0142431.
* Levitin DJ, Chordia P, Menon V. (2012) [Musical rhythm spectra from Bach to Joplin obey a 1/f power law](https://www.pnas.org/content/109/10/3716/). Proceedings of the National Academy of Sciences. 2012;109:3716–20

Histogram plot:

In [ ]:
data = [pink() for _ in range(5000)]
histogram(data)

Listen to  'continuous' pink noise randomness:

In [ ]:
playcontinuous(data, -1, 1)

Listen to  discrete pink noise randomness:

In [ ]:
playdiscrete(data, -1, 1)

### Brown noise

```brown()```

Returns brownish (1/f**2) noise samples between -1 and 1.

Histogram plot:

In [ ]:
data = [brown() for _ in range(5000)]
histogram(data)

Listen to 'continuous' brown noise randomness:

In [ ]:
playcontinuous(data, -1, 1)

Listen to  discrete brown noise randomness:

In [ ]:
playdiscrete(data, -1, 1)

## Special purpose random generators

### ```between(a, b)```

Returns a random value between a and b (exclusive). An int is returned if both a and b are ints, otherwise a float is returned. 

In [ ]:
[between(20, 30) for _ in range(10)]

In [ ]:
[between(20.0, 30) for _ in range(10)]

### ```odds(prob, true, false)```

Returns the value in *true* if a random choice is less than *prob* else returns the value in *false*.

In [ ]:
[odds(.2, "c4", "fs4") for _ in range(10)]

### ```vary(num, pct, shift)```



Returns a random number that deviates from value *num* or list of the same by up to variance (1=100%) according to shift. If shift is None then then value is at the center of what could be returned. Shift "+" places the selected value at the minimum of what could be returned and "-" means that the value is the maximum possible value returned.

In [ ]:
[vary(1, .2) for _ in range(10)]

Variance above value:

In [ ]:
[vary(1, .2, '+') for _ in range(10)]

Variance below value:

In [ ]:

[vary(1, .2, '-') for _ in range(10)]

### The spray() note generator

The musx.paint module provides a `spray()` generator that outputs musical notes who's parameters are determined through controlled random selection.  

This example uses `spray()` to create a short blues-ish piece. The choice of notes and their characterisics are determined by random selection within boundaries determined by the composer, but the sectional organization of the composition is completely determined:

In [ ]:
blues = [0, 3, 5, 6, 7, 10, 12]

score = Score(out=Seq())
s1 = spray(score, duration=.2, rhythm=.2, band=[0, 3, 5], pitch=30, amplitude=0.35, end=36)
s2 = spray(score, duration=.2, rhythm=[-.2, -.4, .2, .2], band=[3, 7, 6], pitch=pick(30, 42), amplitude=0.5, end=25)
s3 = spray(score, duration=.2, rhythm=[-.2, .2, .2], band=blues, pitch=pick(42, 54), instrument=2, end=20)
s4 = spray(score, duration=.2, rhythm=[-.6, .4, .4], band=blues, pitch=66, amplitude=0.4, end=15)
s5 = spray(score, duration=.2, rhythm=.2, band=[0, 3, 5], pitch=30, amplitude=0.5, end=10)
s6 = spray(score, duration=.2, rhythm=[-.2, -.4, .2, .2], band=[3, 7, 6], pitch=pick(30, 42), amplitude=0.8, end=10)
s7 = spray(score, duration=.2, rhythm=[-.2, .2, .2], band=blues, pitch=pick(42, 54), instrument=2, end=10)
s8 = spray(score, duration=.2, rhythm=[-.6, .4, .4], band=blues, pitch=66, amplitude=0.6, end=10)
s9 = spray(score, duration=.2, rhythm=.2, band=blues, pitch=66, amplitude=0.4, end=6)

score.compose([[0, s1], [5, s2], [10, s3], [15, s4], [37, s5], 
               [37, s6], [37, s7], [37,s8], [47,s9]])

file = MidiFile("randomness.mid", score.out).write()
print(f"Wrote '{file.pathname}'")
playfile(file.pathname)